In [1]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
from PIL import Image
import math

In [2]:
l = cv.imread('D:/Disparity-Map-Generator/StereoMatchingTestings/Reindeer/view1.png', cv.IMREAD_GRAYSCALE)
r = cv.imread('D:/Disparity-Map-Generator/StereoMatchingTestings/Reindeer/view5.png', cv.IMREAD_GRAYSCALE)

In [3]:
window_size = 3
min_disp = 0
num_disp = 256
stereo = cv.StereoSGBM_create(minDisparity = 0,
                                numDisparities = num_disp,
                                blockSize = window_size,
                                P1=8 * 3 * window_size ** 2,
                                P2=32 * 3 * window_size ** 2,
                                disp12MaxDiff = 1,
                                uniquenessRatio = 10,
                                speckleWindowSize = 150,
                                speckleRange = 1
                                )

In [4]:
disp_map = stereo.compute(l, r).astype(np.float32) / 16.0
disp_map = (disp_map - min_disp) / num_disp

In [5]:
count = 0
for j in range(disp_map.shape[0]):
    if disp_map[0, j] > 0:
        count = j
        break
disp_map_ = np.zeros_like(disp_map)
for i in range(disp_map.shape[0]):

    for j in range(count, disp_map.shape[1] - 1):
        if disp_map[i, j] <= 0:
            left = right = j
            for k in range(j, 1, -1):
                if disp_map[i][k] > 0:
                    left = k
                    break
            for k in range(j, len(disp_map[i])-1):
                if disp_map[i][k] > 0:
                    right = k
                    break
            disp_map_[i, j] = (disp_map[i, left - 1] + disp_map[i, right + 1]) / 2
        else:
            disp_map_[i, j] = disp_map[i, j]

In [6]:
for i in range(10):
    disp_map_ = cv.medianBlur(disp_map_, 3)

In [7]:
plt.imsave('SGBM.png', disp_map_)
disparity_SGBM = cv.imread('SGBM.png')
disparity_SGBM = cv.cvtColor(disparity_SGBM, cv.COLOR_BGR2GRAY)

for row in range(len(disparity_SGBM)):
    for col in range(len(disparity_SGBM[0])):
        disparity_SGBM[row][col] += 2
cv.imwrite('D:/Disparity-Map-Generator/PSNR_Assignment2/4.0 output/Reindeer/disp1.png', disparity_SGBM)

True